<div class="alert alert-info">
  <h1>Pattern Recognition</h1>
  <h3><font color="red">Transformers for Pattern Recognition (HW3)</font></h3>
  <h5>Teaching Assistants: </h5>

*   <a href="https://telegram.me/visionlover">Seyed Mohammad Mousavi</a>
*   <a href="https://telegram.me/Peyman_syh">Peyman attartsayah</a>


</div>

In [1]:
# @title Stdents information

first_name = 'Mohammad' # @param {type:"string", placeholder:"enter your first name"}
last_name = 'Haghighat' # @param {type:"string", placeholder:"enter your last name"}
student_number = '403722042' # @param {type:"string", placeholder:"enter your student_numbe"}

# part b

in this section, we want to use transformer for classification with prompt engineering

## import libraries

In [2]:
!wget https://projectquest.ir/Transformers_for_Pattern_Recognition/requirements.txt

--2024-12-11 17:41:33--  https://projectquest.ir/Transformers_for_Pattern_Recognition/requirements.txt
Resolving projectquest.ir (projectquest.ir)... 5.9.37.15
Connecting to projectquest.ir (projectquest.ir)|5.9.37.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      27  --.-KB/s    in 0s      

2024-12-11 17:41:34 (8.20 MB/s) - ‘requirements.txt’ saved [27/27]



In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [24]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## prepare dataset and generate prompts

In [5]:
!wget https://projectquest.ir/Transformers_for_Pattern_Recognition/all-data.csv

--2024-12-11 17:42:28--  https://projectquest.ir/Transformers_for_Pattern_Recognition/all-data.csv
Resolving projectquest.ir (projectquest.ir)... 5.9.37.15
Connecting to projectquest.ir (projectquest.ir)|5.9.37.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 672006 (656K) [text/csv]
Saving to: ‘all-data.csv’

all-data.csv        100%[===================>] 656.26K   894KB/s    in 0.7s    

2024-12-11 17:42:30 (894 KB/s) - ‘all-data.csv’ saved [672006/672006]



In [6]:
filename = "/content/all-data.csv"
df = pd.read_csv(filename, encoding='latin-1', names=["sentiment", "text"])

positive_samples = df[df['sentiment'] == 'positive'].sample(n=600, random_state=10)
negative_samples = df[df['sentiment'] == 'negative'].sample(n=600, random_state=10)
neutral_samples = df[df['sentiment'] == 'neutral'].sample(n=600, random_state=10)

X_train = [positive_samples.iloc[:300], negative_samples.iloc[:300], neutral_samples.iloc[:300]]
X_test = [positive_samples.iloc[300:], negative_samples.iloc[300:], neutral_samples.iloc[300:]]

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)
X_train = X_train.reset_index(drop=True)
def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = """.strip()

y_true = X_test.sentiment
X_test_s2 = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])


## evaluation

In [7]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

## get lama3.1-8B

In [8]:
!git-lfs clone https://huggingface.co/mlabonne/Hermes-3-Llama-3.1-8B-lorablated

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'Hermes-3-Llama-3.1-8B-lorablated'...
remote: Enumerating objects: 37, done.
remote: Total 37 (delta 0), reused 0 (delta 0), pack-reused 37 (from 1)
Unpacking objects: 100% (37/37), 2.24 MiB | 2.17 MiB/s, done.


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


## prediction

In [10]:
!rm /content/Hermes-3-Llama-3.1-8B-lorablated/tokenizer_config.json
!wget https://projectquest.ir/Transformers_for_Pattern_Recognition/tokenizer_config.json
!mv /content/tokenizer_config.json /content/Hermes-3-Llama-3.1-8B-lorablated/tokenizer_config.json

--2024-12-11 17:51:46--  https://projectquest.ir/Transformers_for_Pattern_Recognition/tokenizer_config.json
Resolving projectquest.ir (projectquest.ir)... 5.9.37.15
Connecting to projectquest.ir (projectquest.ir)|5.9.37.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50007 (49K) [application/json]
Saving to: ‘tokenizer_config.json’

tokenizer_config.js 100%[===================>]  48.83K   167KB/s    in 0.3s    

2024-12-11 17:51:47 (167 KB/s) - ‘tokenizer_config.json’ saved [50007/50007]



In [25]:
model_name = "/content/Hermes-3-Llama-3.1-8B-lorablated"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [13]:
y_pred = predict(X_test_s2, model, tokenizer)

  0%|          | 0/900 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 900/900 [04:08<00:00,  3.62it/s]


In [14]:
evaluate(y_true, y_pred)

Accuracy: 0.708
Accuracy for label 0: 0.910
Accuracy for label 1: 0.677
Accuracy for label 2: 0.537

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.91      0.79       300
           1       0.64      0.68      0.66       300
           2       0.82      0.54      0.65       300

    accuracy                           0.71       900
   macro avg       0.72      0.71      0.70       900
weighted avg       0.72      0.71      0.70       900


Confusion Matrix:
[[273  27   0]
 [ 62 203  35]
 [ 52  87 161]]


# part c (Fine-tuning with Lora)

in this section, we want to fine-tune transformer with Llama 2 for classification

In [15]:
remaining_data = df[~df.index.isin(pd.concat([X_train , X_test] ,axis=0).index)]

positive_eval = remaining_data[remaining_data['sentiment'] == 'positive'].sample(n=50, random_state=10)
negative_eval = remaining_data[remaining_data['sentiment'] == 'negative'].sample(n=50, random_state=10)
neutral_eval = remaining_data[remaining_data['sentiment'] == 'neutral'].sample(n=50, random_state=10)
eval_data = pd.concat([positive_eval, negative_eval, neutral_eval])
X_eval = eval_data.reset_index(drop=True)
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["text"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [16]:
output_dir="trained_weigths"
# Note: Use this parameters:
# lora_alpha=16, lora_dropout=0.1, r=64, bias="none", target_modules="all-linear", task_type="CAUSAL_LM",

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)



training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)


trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, dataset_kwargs. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Tran

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
0,0.928200,0.873886
2,0.512900,0.756812


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=336, training_loss=0.7805818205788022, metrics={'train_runtime': 2643.5043, 'train_samples_per_second': 1.021, 'train_steps_per_second': 0.127, 'total_flos': 1.0159126312771584e+16, 'train_loss': 0.7805818205788022, 'epoch': 2.986666666666667})

In [18]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('trained_weigths/tokenizer_config.json',
 'trained_weigths/special_tokens_map.json',
 'trained_weigths/tokenizer.json')

In [19]:
import gc

del [model, tokenizer, peft_config, trainer, train_data, eval_data, bnb_config, training_arguments]
del [df, X_train, X_eval]
del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]


In [20]:
for _ in range(100):
    torch.cuda.empty_cache()
    gc.collect()

In [26]:
from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     quantization_config=bnb_config
     return_dict=False,
     device_map=device,
    )

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./merged_model")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.json')

In [27]:
y_pred = predict(X_test, merged_model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 0/900 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 900/900 [07:11<00:00,  2.09it/s]

Accuracy: 0.720
Accuracy for label 0: 0.907
Accuracy for label 1: 0.710
Accuracy for label 2: 0.543

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.91      0.80       300
           1       0.65      0.71      0.68       300
           2       0.84      0.54      0.66       300

    accuracy                           0.72       900
   macro avg       0.74      0.72      0.71       900
weighted avg       0.74      0.72      0.71       900


Confusion Matrix:
[[272  28   0]
 [ 57 213  30]
 [ 49  88 163]]
